In [ ]:
import urllib.request
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 논문 검색 결과 크롤링하기

In [ ]:
# https://www.kci.go.kr/kciportal/po/search/poArtiSearList.kci # 검색 키워드는 인권

* F12 클릭 후 Network 옵션을 선택한 다음 F5를 클릭하여 새로고침한 뒤 poArtiSearList.kci 라는 파일을 클릭 (F212 클릭이 막힐 경우 마우스 우클릭 후 검사 클릭하면 우측의 개발자 탭이 표시됨)
* 해당 파일을 클릭 후 Payload 항목을 선택
* Payload 의 Form data 를 드래그하여 챗봇을 활용하여 아래와 같은 {key: value} 형태로 전환

In [ ]:
#### 아래를 선택하고 맨 하단의 체크항목 내 재검색 클릭
# 주제분류: 인문학, 사회과학, 예술체육, 복합학
# 재단등재구분: 우수등재, KCI 등재, KCI 후보
# 발행년도: 2024
# 논문구분: 학술지

param = {'poSearchBean.searType': 'thesis',
         'poSearchBean.conditionList': 'KEYALL',
         'poSearchBean.keywordList': '인권',
         'poSearchBean.pubiStYr': '',
         'poSearchBean.pubiStMt': '',
         'poSearchBean.pubiEndYr': '',
         'poSearchBean.pubiEndMt': '',
         'poSearchBean.sortName': 'SCORE',
         'poSearchBean.sortDir': 'desc',
         'poSearchBean.startPg': '1',
         'poSearchBean.docsCount': '50',
         'poSearchBean.printKeyword': '인권',
         'poSearchBean.keywordRecommend': 'N',
         'poSearchBean.userRecommend': 'N',
         'poSearchBean.isAdvanceSearch': 'false',
         'poSearchBean.queryText': '',
         'poSearchBean.resultForm': 'Y',
         'poSearchBean.filter1': ['A', 'B', 'G', 'H'],
         'poSearchBean.filter3': ['09', '02', '03'],
         'poSearchBean.filter4': ['2024'],
         'poSearchBean.filter5': '01'
         }

req = requests.post('https://www.kci.go.kr/kciportal/po/search/poArtiSearList.kci', data = param)
print(req.text)

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">























<html xmlns="http://www.w3.org/1999/xhtml" lang="ko" xml:lang="ko">
<head>
    <title>KCI 국내학술지 인용색인 정보 포털입니다.</title>
    <link rel="icon" href="/favicon.ico" type="image/x-icon"/>
	<link rel="shortcut icon" href="/favicon.ico" type="image/x-icon"/>
    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8"/>
    <meta http-equiv="X-UA-Compatible" content="IE=edge" />
	
	

	<link type="text/css" href="/kciportal/resources/newkci/css/jquery.mCustomScrollbar.css?v=20210603" rel="stylesheet" />

	<link type="text/css" href="/kciportal/resources/newkci/css/fontawesome-all.min.css" rel="stylesheet" />
	<link type="text/css" href="/kciportal/resources/newkci/css/common.css?v=20250522" rel="stylesheet" />
	<link type="text/css" href="/kciportal/resources/newkci/css/base.css?v=20250522_1" rel="stylesheet" />
	<link type="text/css" href="/

In [ ]:
soup = BeautifulSoup(req.text, 'html.parser', from_encoding='utf-8')

# A B : A의 하위 단계 전체에서 B를 찾음(A의 하위 단계 어디엔가 B가 있을 경우 이를 지정하여 출력함)
# A > B: A에서 한 단계만 내려간 상태에서 B 태그를 찾음(즉, > 로 관계를 표현한 상태에서 바로 아랫 단계에 B가 없을 경우 결과물이 출력되지 않음)

# 제목 태그의 위치 추정해보기
content = soup.select('div.search-answer-list > form#poArtiSearList > table.search-answer-tbl a.subject')
content

/usr/local/lib/python3.12/dist-packages/bs4/__init__.py:341: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn(


[<a class="subject" href="/kciportal/ci/sereArticleSearch/ciSereArtiView.kci?sereArticleSearchBean.artiId=ART003158864" onclick="fncSaveDetailViewLog('A', 'ART003158864'); fnArtiDetail('ART003158864'); return false;">
 												Who works with human rights at the local level?: Case study of human rights cities in Sweden
 											</a>,
 <a class="subject" href="/kciportal/ci/sereArticleSearch/ciSereArtiView.kci?sereArticleSearchBean.artiId=ART003158866" onclick="fncSaveDetailViewLog('A', 'ART003158866'); fnArtiDetail('ART003158866'); return false;">
 												Human Rights Defender Criminalization and Civic Space Repression as Challenges to Human Rights Cities Implementation: A Case Study of Indonesia
 											</a>,
 <a class="subject" href="/kciportal/ci/sereArticleSearch/ciSereArtiView.kci?sereArticleSearchBean.artiId=ART003094879" onclick="fncSaveDetailViewLog('A', 'ART003094879'); fnArtiDetail('ART003094879'); return false;">
 												Human Rights Discourse as A Secula

In [ ]:
content[10]

<a class="subject" href="/kciportal/ci/sereArticleSearch/ciSereArtiView.kci?sereArticleSearchBean.artiId=ART003078894" onclick="fncSaveDetailViewLog('A', 'ART003078894'); fnArtiDetail('ART003078894'); return false;">
												인공지능과 <em><em>인권</em></em>
</a>

In [ ]:
# 추가 크롤링을 위한 논문별 링크 생성하기

articles = soup.select('div.search-answer-list a.subject')       # 'div.search-answer-list' 내의 모든 <a class="subject"> 태그를 선택

for article in articles:    # 선택된 각 <a> 태그에 대해 반복
    article = 'https://www.kci.go.kr' + article['href']     # 상대 경로(href)를 절대 경로로 변환하여 전체 URL 구성
    print(article)      # 완성된 논문/콘텐츠 URL 출력

https://www.kci.go.kr/kciportal/ci/sereArticleSearch/ciSereArtiView.kci?sereArticleSearchBean.artiId=ART003158864
https://www.kci.go.kr/kciportal/ci/sereArticleSearch/ciSereArtiView.kci?sereArticleSearchBean.artiId=ART003158866
https://www.kci.go.kr/kciportal/ci/sereArticleSearch/ciSereArtiView.kci?sereArticleSearchBean.artiId=ART003094879
https://www.kci.go.kr/kciportal/ci/sereArticleSearch/ciSereArtiView.kci?sereArticleSearchBean.artiId=ART003104926
https://www.kci.go.kr/kciportal/ci/sereArticleSearch/ciSereArtiView.kci?sereArticleSearchBean.artiId=ART003055103
https://www.kci.go.kr/kciportal/ci/sereArticleSearch/ciSereArtiView.kci?sereArticleSearchBean.artiId=ART003078622
https://www.kci.go.kr/kciportal/ci/sereArticleSearch/ciSereArtiView.kci?sereArticleSearchBean.artiId=ART003094131
https://www.kci.go.kr/kciportal/ci/sereArticleSearch/ciSereArtiView.kci?sereArticleSearchBean.artiId=ART003098996
https://www.kci.go.kr/kciportal/ci/sereArticleSearch/ciSereArtiView.kci?sereArticleSearc

In [ ]:
# 논문의 제목 텍스트 추출하기

articles = soup.select('div.search-answer-list a.subject')  # 'div.search-answer-list' 내의 <a class="subject"> 태그들을 모두 선택

for article in articles:     # 각 <a> 태그에 대해 반복
    text = article.get_text().strip()    # 태그 내 텍스트를 추출하고 앞뒤 공백 제거
    print(text)  # 추출된 텍스트를 출력

Who works with human rights at the local level?: Case study of human rights cities in Sweden
Human Rights Defender Criminalization and Civic Space Repression as Challenges to Human Rights Cities Implementation: A Case Study of Indonesia
Human Rights Discourse as A Secular Religion : A Christian Critique
A Corpus-Based Comparative Analysis of Newspaper Coverage on China’s Human Rights Issues
인권정책기본법의 제정과 인권
인권교육 경험과 인권의식, 인권태도의 관계에 관한 연구 - 2022년 인권의식실태조사를 중심으로 -
Revisiting Carter’s Policy on Korea : Troop Withdrawal, National Security, and Human Rights
The Globalization of North Korean Human Rights:Potential Roles for Applied Anthropology and Education
국가와 국민의 인권 – 제3세대 인권을 중심으로 –
국가인권위원회 진정절차에서 진정원인사실과 인권침해수론(人權侵害數論)에 관한 소고
인공지능과 인권
Applying Judicial and Non-Judicial Approaches for Human Rights Accountability in North Korea
대학 인권센터 인권교육의 현황과 과제 -고등교육법 개정 이후 인권교육 개선방안을 중심으로-
청년세대 인권태도에 영향을 미치는 요인 연구 : 인권접촉수준과 인권옹호활동을 중심으로
인권침해에서 벗어나는 길에서 인권 실천을 만나다: 정신질환자 인권이 침해되는 경험에 기반한 정신장애인의 인권실천의 의

In [ ]:
# 소스코드 내의 논문의 메타데이터 위치 추정하기

metas = soup.select('div.search-answer-list ul.nopm.floats.subject-info')
metas[0]

<ul class="nopm floats subject-info">
<li>
<a href="/kciportal/po/citationindex/poCretDetail.kci?citationBean.cretId=CRT003202765&amp;citationBean.artiId=ART003158864">
															Elizaveta Tuneva
														</a>
<!-- 2024.04.17 추가 -->
</li>
<li>
<a href="/kciportal/po/search/poInsiSearReinView.kci?insiGeneInfoBean.insiId=INS000003145">
															5.18연구소
														</a>
</li>
<li>
<a href="/kciportal/ci/seriesSearch/ciSereInfoView.kci?sereSearBean.insiId=INS000003145&amp;sereSearBean.sereId=001616">
	                                                	민주주의와 <em><em>인권</em></em>
</a>
</li>
<li>
<a href="/kciportal/po/search/poSereArtiList.kci?sereId=001616&amp;volIsseId=VOL000177771">
														24(4)
													</a>
</li>
<li>pp.289~302</li>
<li>2024.12</li>
<li>기타사회과학일반</li>
</ul>

In [ ]:
# To metadata the field from articles

metas = soup.select('div.search-answer-list ul.nopm.floats.subject-info')

for meta in metas:
    field = meta.select('li')[-1].get_text().strip()
    date = meta.select('li')[-2].get_text().strip()
    journal = meta.select('li')[-5].get_text().strip()
    org = meta.select('li')[-6].get_text().strip()
    print(f'{field}\t', f'{date}\t', f'{journal}\t', f'{org}')

기타사회과학일반	 2024.12	 민주주의와 인권	 5.18연구소
기타사회과학일반	 2024.12	 민주주의와 인권	 5.18연구소
기타인문학	 2024.06	 기독교와 문화	 한국기독교문화연구원
기타인문학	 2024.07	 인문과학연구	 인문과학연구소
법학	 2024.02	 동아법학	 법학연구소
학제간연구	 2024.04	 문화와융합	 인문사회예술융합학회
지역학	 2024.06	 아세아연구	 아세아문제연구원
학제간연구	 2024.06	 동북아 문화연구	 동북아시아문화학회
헌법	 2024.12	 헌법학연구	 한국헌법학회
법학	 2024.06	 법학논총	 법학연구소
비교법학	 2024.04	 법학논총	 법학연구원
인권/국제인권법	 2024.12	 인권연구	 한국인권학회
사회과학일반	 2024.08	 법과인권교육연구	 한국 법과인권교육학회
학제간연구	 2024.09	 한국과 세계	 한국국회학회
정책학일반	 2024.10	 Crisisonomy	 (사)위기관리이론과실천
기독교신학	 2024.09	 신학사상	 신학사상연구소
인권/국제인권법	 2024.06	 인권연구	 한국인권학회
기타사회과학일반	 2024.12	 민주주의와 인권	 5.18연구소
법학	 2024.05	 이화젠더법학	 젠더법학연구소
체육교육학	 2024.04	 한국체육교육학회지	 한국체육교육학회
학제간연구	 2024.09	 한국과 세계	 한국국회학회
문화/사회/보건복지정책	 2024.06	 보건과 복지	 한국보건복지학회
사회과학일반	 2024.04	 법과인권교육연구	 한국 법과인권교육학회
법학	 2024.11	 민주법학	 민주주의법학연구회
사회과학일반	 2024.08	 법과인권교육연구	 한국 법과인권교육학회
법학	 2024.02	 법이론실무연구	 사단법인 한국법이론실무학회
법학	 2024.03	 법학연구	 한국법학회
기타법학	 2024.04	 法學論文集	 법학연구원
법학	 2024.05	 서울법학	 서울시립대학교 법학연구소
헌법	 2024.12	 헌법학연구	 한국헌법학회
인권/국제인권법	 2024.12	

In [ ]:
# Test to first page

TITLE_OF_ARTICLE = []
URL_OF_ARTICLE = []
DATE_OF_ARTICLE = []
FIELD_OF_ARTICLE = [] # 분야
JOURNAL_OF_ARTICLE = [] # 학술지명
ORG_OF_ARTICLE = [] # 학술지 편찬기관

param = {'poSearchBean.searType': 'thesis',
         'poSearchBean.conditionList': 'KEYALL',
         'poSearchBean.keywordList': '인권',
         'poSearchBean.pubiStYr': '',
         'poSearchBean.pubiStMt': '',
         'poSearchBean.pubiEndYr': '',
         'poSearchBean.pubiEndMt': '',
         'poSearchBean.sortName': 'SCORE',
         'poSearchBean.sortDir': 'desc',
         'poSearchBean.startPg': '1',
         'poSearchBean.docsCount': '50',
         'poSearchBean.printKeyword': '인권',
         'poSearchBean.keywordRecommend': 'N',
         'poSearchBean.userRecommend': 'N',
         'poSearchBean.isAdvanceSearch': 'false',
         'poSearchBean.queryText': '',
         'poSearchBean.resultForm': 'Y',
         'poSearchBean.filter1': ['A', 'B', 'G', 'H'],
         'poSearchBean.filter3': ['09', '02', '03'],
         'poSearchBean.filter4': ['2024'],
         'poSearchBean.filter5': '01'
         }

req = requests.post('https://www.kci.go.kr/kciportal/po/search/poArtiSearList.kci',
                    data = param,
                    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0'})
soup = BeautifulSoup(req.text, 'html.parser', from_encoding='utf-8')

articles = soup.select('div.search-answer-list a.subject')
for article in articles:

    text = article.get_text().strip()
    TITLE_OF_ARTICLE.append(text)

    article = 'https://www.kci.go.kr' + article['href']
    URL_OF_ARTICLE.append(article)

metas = soup.select('div.search-answer-list ul.nopm.floats.subject-info')
for meta in metas:
    field = meta.select('li')[-1].get_text().strip()
    FIELD_OF_ARTICLE.append(field)

    date = meta.select('li')[-2].get_text().strip()
    DATE_OF_ARTICLE.append(date)

    journal = meta.select('li')[-5].get_text().strip()
    JOURNAL_OF_ARTICLE.append(journal)

    org = meta.select('li')[-6].get_text().strip()
    ORG_OF_ARTICLE.append(org)

test = pd.DataFrame(list(zip(TITLE_OF_ARTICLE, URL_OF_ARTICLE, DATE_OF_ARTICLE, FIELD_OF_ARTICLE, JOURNAL_OF_ARTICLE, ORG_OF_ARTICLE)), columns =['Title', 'Url', 'Date', 'Field', 'Journal', 'Org'])
test

/usr/local/lib/python3.11/dist-packages/bs4/__init__.py:339: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn(


,Title,Url,Date,Field,Journal,Org
0,Who works with human rights at the local level?: Case study of human rights cities in Sweden,https://www.kci.go.kr/kciportal/ci/sereArticleSearch/ciSereArtiView.kci?sereArticleSearchBean.artiId=ART003158864,2024.12,기타사회과학일반,민주주의와 인권,5.18연구소
1,Human Rights Defender Criminalization and Civic Space Repression as Challenges to Human Rights Cities Implementation: A Case Study of Indonesia,https://www.kci.go.kr/kciportal/ci/sereArticleSearch/ciSereArtiView.kci?sereArticleSearchBean.artiId=ART003158866,2024.12,기타사회과학일반,민주주의와 인권,5.18연구소
2,Civic Space Under Pressure: The Challenges Posed by National Strategic Projects on Human Rights Cities in Indonesia,https://www.kci.go.kr/kciportal/ci/sereArticleSearch/ciSereArtiView.kci?sereArticleSearchBean.artiId=ART003158865,2024.12,기타사회과학일반,민주주의와 인권,5.18연구소
3,대학 인권센터 인권교육의 현황과 과제 -고등교육법 개정 이후 인권교육 개선방안을 중심으로-,https://www.kci.go.kr/kciportal/ci/sereArticleSearch/ciSereArtiView.kci?sereArticleSearchBean.artiId=ART003117355,2024.08,사회과학일반,법과인권교육연구,한국 법과인권교육학회
4,Applying Judicial and Non-Judicial Approaches for Human Rights Accountability in North Korea,https://www.kci.go.kr/kciportal/ci/sereArticleSearch/ciSereArtiView.kci?sereArticleSearchBean.artiId=ART003152609,2024.12,인권/국제인권법,인권연구,한국인권학회
5,예비 초등교사의 인권교육 강좌 수강 경험이 인권판단에 주는 영향 분석,https://www.kci.go.kr/kciportal/ci/sereArticleSearch/ciSereArtiView.kci?sereArticleSearchBean.artiId=ART003117354,2024.08,사회과학일반,법과인권교육연구,한국 법과인권교육학회
6,어린이 인권도서를 활용한 교수자의 인권교육 경험의 의미 탐구,https://www.kci.go.kr/kciportal/ci/sereArticleSearch/ciSereArtiView.kci?sereArticleSearchBean.artiId=ART003080052,2024.04,사회과학일반,법과인권교육연구,한국 법과인권교육학회
7,스포츠인권 사각지대에 관한 소고,https://www.kci.go.kr/kciportal/ci/sereArticleSearch/ciSereArtiView.kci?sereArticleSearchBean.artiId=ART003094836,2024.06,기타사회과학일반,민주주의와 인권,5.18연구소
8,A Corpus-Based Comparative Analysis of Newspaper Coverage on China’s Human Rights Issues,https://www.kci.go.kr/kciportal/ci/sereArticleSearch/ciSereArtiView.kci?sereArticleSearchBean.artiId=ART003104926,2024.07,기타인문학,인문과학연구,인문과학연구소
9,Human Rights Discourse as A Secular Religion : A Christian Critique,https://www.kci.go.kr/kciportal/ci/sereArticleSearch/ciSereArtiView.kci?sereArticleSearchBean.artiId=ART003094879,2024.06,기타인문학,기독교와 문화,한국기독교문화연구원


In [ ]:
# To gather all results of title texts and hyperlinks

def get_link_from_news_title(page_num, URL):
    for i in range(page_num):

        param = {'poSearchBean.searType': 'thesis',
                'poSearchBean.conditionList': 'KEYALL',
                'poSearchBean.keywordList': '인권',
                'poSearchBean.pubiStYr': '',
                'poSearchBean.pubiStMt': '',
                'poSearchBean.pubiEndYr': '',
                'poSearchBean.pubiEndMt': '',
                'poSearchBean.sortName': 'SCORE',
                'poSearchBean.sortDir': 'desc',
                'poSearchBean.startPg': str(i + 1), # 여기서 기존의 웹크롤링 코드에서 페이지 카운트를 발동시키는 방식을 적용함
                'poSearchBean.docsCount': '50',
                'poSearchBean.printKeyword': '인권',
                'poSearchBean.keywordRecommend': 'N',
                'poSearchBean.userRecommend': 'N',
                'poSearchBean.isAdvanceSearch': 'false',
                'poSearchBean.queryText': '',
                'poSearchBean.resultForm': 'Y',
                'poSearchBean.filter1': ['A', 'B', 'G', 'H'],
                'poSearchBean.filter3': ['09', '02', '03'],
                'poSearchBean.filter4': ['2024'],
                'poSearchBean.filter5': '01'
                }

        req = requests.post(URL,
                            data = param,
                            headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0'})
        soup = BeautifulSoup(req.text, 'html.parser', from_encoding='utf-8')

        articles = soup.select('div.search-answer-list a.subject')
        for article in articles:

            text = article.get_text().strip()
            TITLE_OF_ARTICLE.append(text)

            article = 'https://www.kci.go.kr' + article['href']
            URL_OF_ARTICLE.append(article)

        metas = soup.select('div.search-answer-list ul.nopm.floats.subject-info')
        for meta in metas:
            field = meta.select('li')[-1].get_text().strip()
            FIELD_OF_ARTICLE.append(field)

            date = meta.select('li')[-2].get_text().strip()
            DATE_OF_ARTICLE.append(date)

            journal = meta.select('li')[-5].get_text().strip()
            JOURNAL_OF_ARTICLE.append(journal)

            org = meta.select('li')[-6].get_text().strip()
            ORG_OF_ARTICLE.append(org)

        if i % 10 == 0:
            time.sleep(2)
        if i % 10 == 0:
            print(f'{i*10}nd article complete')

In [ ]:
# 페이지 개수 추정
# 위의 parameter 에서 페이지당 제시되는 논문의 개수가 50개이므로(default) 이를 기준으로

def get_total_num_of_article(num):
    num = int(num)  # 정수형으로 변환
    if(num % 50 >= 1):
        page_num = int((num / 50) + 1)
    else:
        page_num = int(num / 50)
    return page_num

get_total_num_of_article(502)

11

In [ ]:
# Let's test the code

TITLE_OF_ARTICLE = []
URL_OF_ARTICLE = []
DATE_OF_ARTICLE = []
FIELD_OF_ARTICLE = []
JOURNAL_OF_ARTICLE = []
ORG_OF_ARTICLE = []

get_link_from_news_title(11, 'https://www.kci.go.kr/kciportal/po/search/poArtiSearList.kci') # 크롤링이 필요한 페이지 개수와 원본 링크 입력

/usr/local/lib/python3.12/dist-packages/bs4/__init__.py:341: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn(


0nd article complete
100nd article complete


In [ ]:
df = pd.DataFrame(list(zip(TITLE_OF_ARTICLE, URL_OF_ARTICLE, DATE_OF_ARTICLE, FIELD_OF_ARTICLE, JOURNAL_OF_ARTICLE, ORG_OF_ARTICLE)), columns =['Title', 'Url', 'Date', 'Field', 'Journal', 'Org'])
df

,Title,Url,Date,Field,Journal,Org
0,Who works with human rights at the local level...,https://www.kci.go.kr/kciportal/ci/sereArticle...,2024.12,기타사회과학일반,민주주의와 인권,5.18연구소
1,Human Rights Defender Criminalization and Civi...,https://www.kci.go.kr/kciportal/ci/sereArticle...,2024.12,기타사회과학일반,민주주의와 인권,5.18연구소
2,Human Rights Discourse as A Secular Religion :...,https://www.kci.go.kr/kciportal/ci/sereArticle...,2024.06,기타인문학,기독교와 문화,한국기독교문화연구원
3,A Corpus-Based Comparative Analysis of Newspap...,https://www.kci.go.kr/kciportal/ci/sereArticle...,2024.07,기타인문학,인문과학연구,인문과학연구소
4,인권정책기본법의 제정과 인권,https://www.kci.go.kr/kciportal/ci/sereArticle...,2024.02,법학,동아법학,법학연구소
...,...,...,...,...,...,...
497,비자발적 실직 청년들의 재취업 과정에 대한 연구: 근거이론을 중심으로,https://www.kci.go.kr/kciportal/ci/sereArticle...,2024.09,진로교육,進路敎育硏究,한국진로교육학회
498,IPA 분석을 활용한 자치경찰 사무분석: 생활안전 사무를 중심으로,https://www.kci.go.kr/kciportal/ci/sereArticle...,2024.08,경찰행정,한국치안행정논집,한국치안행정학회
499,한국 성소수자 연구 동향과 과제 - 체계적 문헌고찰 및 텍스트 분석,https://www.kci.go.kr/kciportal/ci/sereArticle...,2024.09,사회학,경제와 사회,비판사회학회
500,한국판 소셜미디어 사용 메타인지 척도 번안 및 타당화,https://www.kci.go.kr/kciportal/ci/sereArticle...,2024.04,학제간연구,한국콘텐츠학회 논문지,한국콘텐츠학회


In [ ]:
# DataFrame을 CSV 파일로 저장
df.to_csv("/content/drive/MyDrive/CLASS/202502_NLP/KCI_인권.csv",     # 저장할 파일 경로 (Google Drive 내 지정 경로)
    index=False,    # 행 번호(인덱스)를 저장하지 않음
    encoding='utf-8-sig'    # 인코딩 방식: 한글 깨짐 방지를 위해 'utf-8-sig' 사용
)

## 논문 초록 추출하기

In [ ]:
# Collect one article

url = urllib.request.Request('https://www.kci.go.kr/kciportal/ci/sereArticleSearch/ciSereArtiView.kci?sereArticleSearchBean.artiId=ART003158864', # 혹은 URL_OF_ARTICLE[0] 이나 df['Url][0]
                             headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.85 Safari/537.36'})
html = urllib.request.urlopen(url)
soup = BeautifulSoup(html, 'html.parser', from_encoding='utf-8')

content = soup.select('div.innerBox.open')
content = content[0].get_text().strip()
content

'Even though human rights cities can be found in all corners of the world, there is no universal definition of or procedure for becoming one. The current paper presents the results of the research project conducted in Sweden in 2023 in both human rights cities of the country, Lund and Piteå. While explaining the process of obtaining the status, a special focus is made on actors and stakeholders involved in this process. The results show that in case of Lund, besides politicians, civil servants, civil society and citizens, that are also present in Piteå, there are important academic institutions, such as RWI and Lund University. However, in both cities it does not seem that citizens are actively involved in this work.\n인권 도시는 세계 곳곳에서 찾아볼 수 있지만, 인권 도시가 되기 위한 보편적 정의나 절차는 존재하지 않는다. 이 논문은 2023년 스웨덴의 두 인권 도시인 룬드시와 피테오시를 대상으로 수행된 연구 결과를 제시한다. 특별히 인권 도시 지위를 획득하는과정을 설명하면서, 이 과정에 관여하는 행위자와 이해관계자들에 초점을 맞춘다. 피테오시에는 정치인, 공무원, 시민사회, 일반시민들이 참여하는 반면, 룬드시의 경우에는 라울 발렌베리 연구소(RWI)와 룬드 대학과 같은 중요한 학술 기관들이 참

In [ ]:
# 국문 초록(논문이 영문으로 작성된 경우 fola가 국문, korAbst 가 영문으로 나타나는 것으로 보임)

url = urllib.request.Request('https://www.kci.go.kr/kciportal/ci/sereArticleSearch/ciSereArtiView.kci?sereArticleSearchBean.artiId=ART003158864',
                             headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.85 Safari/537.36'})
html = urllib.request.urlopen(url)
soup = BeautifulSoup(html, 'html.parser', from_encoding='utf-8')
content = soup.select('div.innerBox.open > p#folaAbst')
content

[<p id="folaAbst" style="white-space: pre-line; letter-spacing: .2pt">인권 도시는 세계 곳곳에서 찾아볼 수 있지만, 인권 도시가 되기 위한 보편적 정의나 절차는 존재하지 않는다. 이 논문은 2023년 스웨덴의 두 인권 도시인 룬드시와 피테오시를 대상으로 수행된 연구 결과를 제시한다. 특별히 인권 도시 지위를 획득하는과정을 설명하면서, 이 과정에 관여하는 행위자와 이해관계자들에 초점을 맞춘다. 피테오시에는 정치인, 공무원, 시민사회, 일반시민들이 참여하는 반면, 룬드시의 경우에는 라울 발렌베리 연구소(RWI)와 룬드 대학과 같은 중요한 학술 기관들이 참여하고있음을 보여준다. 그러나 두 도시 모두에서 일반 시민들이 인권도시 사업에 적극적으로 참여하고 있지 않은 것으로 보인다.</p>]

In [ ]:
# 영문 초록

url = urllib.request.Request('https://www.kci.go.kr/kciportal/ci/sereArticleSearch/ciSereArtiView.kci?sereArticleSearchBean.artiId=ART003158864',
                             headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.85 Safari/537.36'})
html = urllib.request.urlopen(url)
soup = BeautifulSoup(html, 'html.parser', from_encoding='utf-8')
content = soup.select('div.innerBox.open > p#korAbst')
content

[<p id="korAbst" style="white-space: pre-line; letter-spacing: .2pt">Even though human rights cities can be found in all corners of the world, there is no universal definition of or procedure for becoming one. The current paper presents the results of the research project conducted in Sweden in 2023 in both human rights cities of the country, Lund and Piteå. While explaining the process of obtaining the status, a special focus is made on actors and stakeholders involved in this process. The results show that in case of Lund, besides politicians, civil servants, civil society and citizens, that are also present in Piteå, there are important academic institutions, such as RWI and Lund University. However, in both cities it does not seem that citizens are actively involved in this work.</p>]

In [ ]:
# 논문 초록 추출하는 반복문 함수 정의

def get_texts_from_news(articles):
    for i, URL in zip(range(len(articles)), articles):

        LINK = urllib.request.Request(URL, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.85 Safari/537.36'})
        source_code_form_url = urllib.request.urlopen(LINK)
        soup = BeautifulSoup(source_code_form_url, 'html.parser', from_encoding='utf-8')

        # 논문 한글 초록 추출 시도
        try:
            korAbst = soup.select('div.innerBox.open > p#korAbst')
            korAbst = korAbst[0].get_text().strip()
            KABST_OF_ARTICLE.append(korAbst)
        except:
            korAbst = ''
            KABST_OF_ARTICLE.append(korAbst)

        # 논문 영문 초록 추출 시도
        try:
            folaAbst = soup.select('div.innerBox.open > p#folaAbst')
            folaAbst = folaAbst[0].get_text().strip()
            FABST_OF_ARTICLE.append(folaAbst)
        except:
            folaAbst = ''
            FABST_OF_ARTICLE.append(folaAbst)

        # 논문 키워드 추출 시도
        try:
            keyword = soup.select('div.articleBody div.innerBox.open > p')
            keyword = keyword[-1].get_text().strip()
            KEYWORD_OF_ARTICLE.append(keyword)
        except:
            keyword = ''
            KEYWORD_OF_ARTICLE.append(keyword)

        if i % 10 == 0:
            time.sleep(1)
        if i % 100 == 0:
            print(f'{i}nd article complete')

In [ ]:
KABST_OF_ARTICLE = []
FABST_OF_ARTICLE = []
KEYWORD_OF_ARTICLE = []

get_texts_from_news(df['Url'])

0nd article complete
100nd article complete
200nd article complete
300nd article complete
400nd article complete
500nd article complete


In [ ]:
df['KorAbst'] = KABST_OF_ARTICLE
df['FolaAbst'] = FABST_OF_ARTICLE
df['Keyword'] = KEYWORD_OF_ARTICLE

df

,Title,Url,Date,Field,Journal,Org,KorAbst,FolaAbst,Keyword
0,Who works with human rights at the local level...,https://www.kci.go.kr/kciportal/ci/sereArticle...,2024.12,기타사회과학일반,민주주의와 인권,5.18연구소,Even though human rights cities can be found i...,"인권 도시는 세계 곳곳에서 찾아볼 수 있지만, 인권 도시가 되기 위한 보편적 정의나...","인권, 지방 협치, 인권 도시, 스웨덴"
1,Human Rights Defender Criminalization and Civi...,https://www.kci.go.kr/kciportal/ci/sereArticle...,2024.12,기타사회과학일반,민주주의와 인권,5.18연구소,"Since 2015, the concept of human rights cities...",2015년부터 인도네시아에서는 인도네시아 개발에 관한 국제 NGO 포럼(INFID)...,"인권 도시, 인도네시아, 시민 공간, 인권 옹호자, 범죄화, 지방 정부"
2,Human Rights Discourse as A Secular Religion :...,https://www.kci.go.kr/kciportal/ci/sereArticle...,2024.06,기타인문학,기독교와 문화,한국기독교문화연구원,This thesis critically examines the characteri...,본 논문은 세속 종교의 한 형태로서 인권 담론의 특징을 비판적으로검토하고 기독교적 ...,"인권 담론, 세속 종교, 기독교, 독단주의"
3,A Corpus-Based Comparative Analysis of Newspap...,https://www.kci.go.kr/kciportal/ci/sereArticle...,2024.07,기타인문학,인문과학연구,인문과학연구소,This study aims to answer the two questions: W...,이 연구는 두 가지 질문에 초점을 맞추고 있습니다: 미국 미디어와 중국 미디어에서 ...,"인권, 중국, 중국 데일리, 뉴욕 타임스, 토픽 모델링"
4,인권정책기본법의 제정과 인권,https://www.kci.go.kr/kciportal/ci/sereArticle...,2024.02,법학,동아법학,법학연구소,인권에 관한 기본법을 제정하기 위한 시도는 국가인권위원회의 설립에 관한 논의가 이루...,,"Human Rights, Human Rights Policy, Fundamental..."
...,...,...,...,...,...,...,...,...,...
497,비자발적 실직 청년들의 재취업 과정에 대한 연구: 근거이론을 중심으로,https://www.kci.go.kr/kciportal/ci/sereArticle...,2024.09,진로교육,進路敎育硏究,한국진로교육학회,본 연구는 근거이론을 적용하여 청년 실직자들의 재취업 과정을 분석하고 이해하는 데 ...,,"involuntary unemployment, youth, overcoming pr..."
498,IPA 분석을 활용한 자치경찰 사무분석: 생활안전 사무를 중심으로,https://www.kci.go.kr/kciportal/ci/sereArticle...,2024.08,경찰행정,한국치안행정논집,한국치안행정학회,최근 지역사회에서 발생하는 다양한 강력범죄와 안전사고로 인해 주민들의 안전 요구가 ...,,"municipal police, affairs, daily life safety, ..."
499,한국 성소수자 연구 동향과 과제 - 체계적 문헌고찰 및 텍스트 분석,https://www.kci.go.kr/kciportal/ci/sereArticle...,2024.09,사회학,경제와 사회,비판사회학회,이 연구는 한국 성소수자 연구의 동향을 종합적으로 분석해 비규범적 성에 관한 지식과...,,"Sexual and gender minority, sexual and gender ..."
500,한국판 소셜미디어 사용 메타인지 척도 번안 및 타당화,https://www.kci.go.kr/kciportal/ci/sereArticle...,2024.04,학제간연구,한국콘텐츠학회 논문지,한국콘텐츠학회,본 연구는 소셜미디어 사용 메타인지 척도(Meta-cognition about So...,,"Social Media, Meta-cognition, Scale Validation..."


In [ ]:
# DataFrame을 CSV 파일로 저장
df.to_csv("/content/drive/MyDrive/CLASS/202502_NLP/KCI_인권_최종.csv",     # 저장할 파일 경로 (Google Drive 내 지정 경로)
    index=False,    # 행 번호(인덱스)를 저장하지 않음
    encoding='utf-8-sig'    # 인코딩 방식: 한글 깨짐 방지를 위해 'utf-8-sig' 사용
)